#Data

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

#create session
import pyspark
from pyspark.sql import SparkSession


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [2]:
!git clone https://github.com/nicolasmontano/pyspark.git
%cd pyspark/

Cloning into 'pyspark'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
/content/pyspark


In [5]:
data= spark.read.format('csv').option('header','true').load('pyspark/Bank_data.csv')

AttributeError: ignored

In [ ]:
from pyspark.sql.types import *
schema = StructType([
                     StructField('interest_rate',FloatType()),
                     StructField('credit',IntegerType()),
                     StructField('march',IntegerType()),
                     StructField('may',IntegerType()),
                     StructField('previous',IntegerType()),
                     StructField('duration',IntegerType()),
                     StructField('y',StringType()),
                     

])
data=spark.read.csv('pyspark/Bank_data.csv',header=True,schema=schema)
data.dtypes

[('interest_rate', 'float'),
 ('credit', 'int'),
 ('march', 'int'),
 ('may', 'int'),
 ('previous', 'int'),
 ('duration', 'int'),
 ('y', 'string')]

In [ ]:
data.show(7)

+-------------+------+-----+---+--------+--------+---+
|interest_rate|credit|march|may|previous|duration|  y|
+-------------+------+-----+---+--------+--------+---+
|        1.334|     0|    1|  0|       0|     117| no|
|        0.767|     0|    0|  2|       1|     274|yes|
|        4.858|     0|    1|  0|       0|     167| no|
|         4.12|     0|    0|  0|       0|     686|yes|
|        4.856|     0|    1|  0|       0|     157| no|
|        0.899|     0|    0|  1|       0|     126| no|
|         null|     0| null|  0|    null|      84| no|
+-------------+------+-----+---+--------+--------+---+
only showing top 7 rows



#Description

In [ ]:
from pyspark.sql.functions import *
data.describe().show()

print(data.columns)
print(data.count())
print(data.distinct().count())

+-------+------------------+-------------------+-------------------+-------------------+------------------+------------------+----+
|summary|     interest_rate|             credit|              march|                may|          previous|          duration|   y|
+-------+------------------+-------------------+-------------------+-------------------+------------------+------------------+----+
|  count|               517|                518|                517|                518|               517|               518| 518|
|   mean|2.8316634228197244|0.03474903474903475|0.26692456479690524|0.38803088803088803|0.1276595744680851|382.17760617760615|null|
| stddev| 1.876383451206821|0.18332052963359008|0.44278105001455376| 0.8145269438008462|0.3340335709498733|344.29598973814836|null|
|    min|             0.635|                  0|                  0|                  0|                 0|                 9|  no|
|    max|              4.97|                  1|                  1|        

#Operations

In [ ]:
#Create a column
data2=data.withColumn('new_column',when(data.interest_rate.isNull(),'Unknown').otherwise(data.march))
data2.show(4)

+-------------+------+-----+---+--------+--------+---+----------+
|interest_rate|credit|march|may|previous|duration|  y|new_column|
+-------------+------+-----+---+--------+--------+---+----------+
|        1.334|     0|    1|  0|       0|     117| no|         1|
|        0.767|     0|    0|  2|       1|     274|yes|         0|
|        4.858|     0|    1|  0|       0|     167| no|         1|
|         4.12|     0|    0|  0|       0|     686|yes|         0|
+-------------+------+-----+---+--------+--------+---+----------+
only showing top 4 rows



In [ ]:
#drop columns
data2.drop('new_column','may').show(4)

+-------------+------+-----+--------+--------+---+
|interest_rate|credit|march|previous|duration|  y|
+-------------+------+-----+--------+--------+---+
|        1.334|     0|    1|       0|     117| no|
|        0.767|     0|    0|       1|     274|yes|
|        4.858|     0|    1|       0|     167| no|
|         4.12|     0|    0|       0|     686|yes|
+-------------+------+-----+--------+--------+---+
only showing top 4 rows



In [ ]:
#Filter by column
data2=data2.filter(data2.march.isNotNull())
data2.show(7)

+-------------+------+-----+---+--------+--------+---+----------+
|interest_rate|credit|march|may|previous|duration|  y|new_column|
+-------------+------+-----+---+--------+--------+---+----------+
|        1.334|     0|    1|  0|       0|     117| no|         1|
|        0.767|     0|    0|  2|       1|     274|yes|         0|
|        4.858|     0|    1|  0|       0|     167| no|         1|
|         4.12|     0|    0|  0|       0|     686|yes|         0|
|        4.856|     0|    1|  0|       0|     157| no|         1|
|        0.899|     0|    0|  1|       0|     126| no|         0|
|        4.858|     0|    1|  0|       0|      17| no|         1|
+-------------+------+-----+---+--------+--------+---+----------+
only showing top 7 rows



In [ ]:
data.withColumn('interest_rate',data.interest_rate.substr(2,100).cast('float')).show(7) #start from the second
data2.show()

+-------------+------+-----+---+--------+--------+---+
|interest_rate|credit|march|may|previous|duration|  y|
+-------------+------+-----+---+--------+--------+---+
|        0.334|     0|    1|  0|       0|     117| no|
|        0.767|     0|    0|  2|       1|     274|yes|
|        0.858|     0|    1|  0|       0|     167| no|
|         0.12|     0|    0|  0|       0|     686|yes|
|        0.856|     0|    1|  0|       0|     157| no|
|        0.899|     0|    0|  1|       0|     126| no|
|         null|     0| null|  0|    null|      84| no|
+-------------+------+-----+---+--------+--------+---+
only showing top 7 rows



In [ ]:
column_df=data2.dropna().withColumn('new_interest_3',data2.interest_rate.cast('float')).select('new_interest_3') #select a column with a type
column_df.show(2)

+--------------+
|new_interest_3|
+--------------+
|         1.334|
|         0.767|
+--------------+
only showing top 2 rows



In [ ]:
import numpy as np
median=np.median(column_df.collect())
print(median)

1.4529999494552612


In [ ]:
data2=data.withColumn('interest_rate',when(data.interest_rate.isNull(),lit(median)).otherwise(data.interest_rate.cast('float')))
data2.show(2)

+------------------+------+-----+---+--------+--------+---+
|     interest_rate|credit|march|may|previous|duration|  y|
+------------------+------+-----+---+--------+--------+---+
|1.3339999914169312|     0|    1|  0|       0|     117| no|
|0.7670000195503235|     0|    0|  2|       1|     274|yes|
+------------------+------+-----+---+--------+--------+---+
only showing top 2 rows



In [ ]:
data2.withColumn('yes',when(data2.y=='yes',data2.interest_rate).otherwise(0)).show(4)

+------------------+------+-----+---+--------+--------+---+------------------+
|     interest_rate|credit|march|may|previous|duration|  y|               yes|
+------------------+------+-----+---+--------+--------+---+------------------+
|1.3339999914169312|     0|    1|  0|       0|     117| no|               0.0|
|0.7670000195503235|     0|    0|  2|       1|     274|yes|0.7670000195503235|
| 4.857999801635742|     0|    1|  0|       0|     167| no|               0.0|
| 4.119999885559082|     0|    0|  0|       0|     686|yes| 4.119999885559082|
+------------------+------+-----+---+--------+--------+---+------------------+
only showing top 4 rows



In [ ]:
data3=data2.withColumn('yes',when(data2.y=='yes',data2.interest_rate).otherwise(0))
data3.sort(col('yes').desc()).show(4)

+------------------+------+-----+---+--------+--------+---+------------------+
|     interest_rate|credit|march|may|previous|duration|  y|               yes|
+------------------+------+-----+---+--------+--------+---+------------------+
|  4.96999979019165|     0|    0|  0|       0|     846|yes|  4.96999979019165|
|4.9679999351501465|     0|    0|  0|       0|    1311|yes|4.9679999351501465|
|4.9679999351501465|     0|    0|  0|       0|      85|yes|4.9679999351501465|
|4.9670000076293945|     0|    0|  0|       0|    2653|yes|4.9670000076293945|
+------------------+------+-----+---+--------+--------+---+------------------+
only showing top 4 rows



In [ ]:
  data.describe().show()

+-------+------------------+-------------------+-------------------+-------------------+------------------+------------------+----+
|summary|     interest_rate|             credit|              march|                may|          previous|          duration|   y|
+-------+------------------+-------------------+-------------------+-------------------+------------------+------------------+----+
|  count|               517|                518|                517|                518|               517|               518| 518|
|   mean|2.8316634228197244|0.03474903474903475|0.26692456479690524|0.38803088803088803|0.1276595744680851|382.17760617760615|null|
| stddev| 1.876383451206821|0.18332052963359008|0.44278105001455376| 0.8145269438008462|0.3340335709498733|344.29598973814836|null|
|    min|             0.635|                  0|                  0|                  0|                 0|                 9|  no|
|    max|              4.97|                  1|                  1|        

#Null

In [ ]:
data.select([count(when(data[c].isNull(), c)).alias(c) for c in data.columns]).show()

+-------------+------+-----+---+--------+--------+---+
|interest_rate|credit|march|may|previous|duration|  y|
+-------------+------+-----+---+--------+--------+---+
|            1|     0|    1|  0|       1|       0|  0|
+-------------+------+-----+---+--------+--------+---+



In [ ]:
data.filter(data.march.isNull()).show()

+-------------+------+-----+---+--------+--------+---+
|interest_rate|credit|march|may|previous|duration|  y|
+-------------+------+-----+---+--------+--------+---+
|         null|     0| null|  0|    null|      84| no|
+-------------+------+-----+---+--------+--------+---+



In [ ]:
#not null records
print('Not nulls records:{}'.format(data.na.drop().count()))
data.dropna().count()

Not nulls records:517


517

In [ ]:
#not null records by column
print(data.filter(data.interest_rate.isNotNull()).count())
print(data.filter('interest_rate IS NOT NULL').count())

517
517


In [ ]:
data.withColumn('march',when(data.march.isNull(),1).otherwise(data.march)).show(4)

+-------------+------+-----+---+--------+--------+---+
|interest_rate|credit|march|may|previous|duration|  y|
+-------------+------+-----+---+--------+--------+---+
|        1.334|     0|    1|  0|       0|     117| no|
|        0.767|     0|    0|  2|       1|     274|yes|
|        4.858|     0|    1|  0|       0|     167| no|
|         4.12|     0|    0|  0|       0|     686|yes|
+-------------+------+-----+---+--------+--------+---+
only showing top 4 rows



#Duplicates

In [ ]:
data.drop_duplicates().count()

514

#Filter

In [ ]:
print(data.filter(data.y=='no').count())
print(data.filter(data.y.like('%n%')).count())
print(data.filter(data.y.endswith('o')).count())
print(data.filter(data.y.startswith('n')).count())
print(data.filter(data.y.isin('no','yes')).count())

259
259
259
259
518


In [ ]:
data.filter(data.march.between(1,2)).show(2)
data.filter((data.interest_rate>3) & (data.duration>1000)).show(2) #AND
data.filter((data.interest_rate>3) | (data.duration>1000)).show(2) #or

+-------------+------+-----+---+--------+--------+---+
|interest_rate|credit|march|may|previous|duration|  y|
+-------------+------+-----+---+--------+--------+---+
|        1.334|     0|    1|  0|       0|     117| no|
|        4.858|     0|    1|  0|       0|     167| no|
+-------------+------+-----+---+--------+--------+---+
only showing top 2 rows

+-------------+------+-----+---+--------+--------+---+
|interest_rate|credit|march|may|previous|duration|  y|
+-------------+------+-----+---+--------+--------+---+
|         4.12|     0|    0|  1|       0|    1011|yes|
|        4.962|     0|    0|  0|       0|    1130|yes|
+-------------+------+-----+---+--------+--------+---+
only showing top 2 rows

+-------------+------+-----+---+--------+--------+---+
|interest_rate|credit|march|may|previous|duration|  y|
+-------------+------+-----+---+--------+--------+---+
|        4.858|     0|    1|  0|       0|     167| no|
|         4.12|     0|    0|  0|       0|     686|yes|
+-------------+

#Grouping

In [ ]:
import pyspark.sql.functions as sqlfunc
data.groupBy('y').agg(round(sqlfunc.avg('interest_rate')).alias('avg'),
                       sqlfunc.sum('interest_rate').alias('sum'),
                       sqlfunc.min('interest_rate').alias('min'),
                       sqlfunc.max('interest_rate').alias('max')).show()
                       

#SQL

In [ ]:
data.registerTempTable('table')
#df.createOrReplaceTempView('table1')
spark.sql("DESCRIBE table").show()
query1=spark.sql('select concat(interest_rate," + ",y) as sss from table')
query1.show(3)

# Print the tables in the catalog
print(spark.catalog.listTables())

+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|interest_rate|    float|   null|
|       credit|      int|   null|
|        march|      int|   null|
|          may|      int|   null|
|     previous|      int|   null|
|     duration|      int|   null|
|            y|   string|   null|
+-------------+---------+-------+

+-----------+
|        sss|
+-----------+
| 1.334 + no|
|0.767 + yes|
| 4.858 + no|
+-----------+
only showing top 3 rows

[Table(name='table', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [ ]:
# Run the query
flight_counts = spark.sql(query)

# Convert the results to a pandas DataFrame
pd_counts = flight_counts.toPandas()

# Print the head of pd_counts
print(pd_counts.head())

In [ ]:
query = """
SELECT train_id, station, time, diff_min,
SUM(diff_min) OVER (PARTITION BY train_id ORDER BY time) AS running_total
FROM schedule
"""
LEAD(TIME,1) OVER (PARTITION BY train_id ORDER BY time) AS running_total



#Export

In [ ]:
data.write.csv('data.csv')
data.write.json('data.json')

In [ ]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Create my_spark
my_spark = SparkSession.builder.getOrCreate()

# Print my_spark
print(my_spark)

In [ ]:
# Verify SparkContext
print(sc)
# Print Spark version
print(sc.version)


NameError: ignored

In [ ]:
# Run the query
flight_counts = spark.sql(query)

# Convert the results to a pandas DataFrame
pd_counts = flight_counts.toPandas()

# Print the head of pd_counts
print(pd_counts.head())

In [ ]:
data.write.csv('data.csv')
data.write.json('data.json')